In [1]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [22]:
df = pd.read_csv('../../data/processed/processed_data_with_features.csv')
df_retailer2 = df[df['retailer'] == 'retail2']
X = df_retailer2.drop(columns=['quantity', 'retailer'])
y = df_retailer2['quantity']


In [23]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical columns to numeric (one-hot encoding)
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Align columns in case train and test sets have different one-hot encoded columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Ensure all columns are numeric to avoid dtype issues
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)

# Diagnostic: Check for any remaining non-numeric columns
print("Data types in X_train after processing:", X_train.dtypes.value_counts())

Data types in X_train after processing: float64    30
bool        5
int64       4
Name: count, dtype: int64


In [6]:
# Convert all columns in X_train and X_test to float64
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Verify data types
print("Data types in X_train after final conversion:", X_train.dtypes.value_counts())
print("Data types in X_test after final conversion:", X_test.dtypes.value_counts())

with pm.Model() as bayesian_model:
    # Priors
    intercept = pm.Normal('intercept', mu=10, sigma=2)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Define coefficients for each feature
    coefs = pm.Normal('coefs', mu=0, sigma=1, shape=X_train.shape[1])

    # Linear regression model
    y_pred = intercept + pm.math.dot(X_train.values, coefs)

    # Likelihood
    y_obs = pm.Normal('y_obs', mu=y_pred, sigma=sigma, observed=y_train.values)

    # Sampling with reduced draws and chains for speed, using the Metropolis step method
    step = pm.Metropolis()
    trace = pm.sample(draws=500, tune=500, chains=2, step=step)

# Predictions
with bayesian_model:
    post_pred = pm.sample_posterior_predictive(trace)

y_pred = post_pred['y_obs'].mean(axis=0)

# Performance metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Bayesian Model - Mean Squared Error (MSE): {mse}")
print(f"Bayesian Model - Mean Absolute Error (MAE): {mae}")
print(f"Bayesian Model - R-squared (R²): {r2}")



Data types in X_train after final conversion: float64    39
Name: count, dtype: int64
Data types in X_test after final conversion: float64    39
Name: count, dtype: int64


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [intercept]
>Metropolis: [sigma]
>Metropolis: [coefs]


Output()

/home/cian/Ferrero_Tech_Assesment/new_venv/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),


/home/cian/Ferrero_Tech_Assesment/new_venv/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 56 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [y_obs]


Output()

KeyError: 'y_obs'

In [11]:
# Generate posterior predictive samples
with bayesian_model:
    post_pred = pm.sample_posterior_predictive(trace, var_names=['y_obs'])

# Extract and match the length of y_pred to y_test
y_pred_samples = post_pred['y_obs']
y_pred = y_pred_samples.mean(axis=0)[:len(y_test)]

# Performance metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Bayesian Model - Mean Squared Error (MSE): {mse}")
print(f"Bayesian Model - Mean Absolute Error (MAE): {mae}")
print(f"Bayesian Model - R-squared (R²): {r2}")


Sampling: [y_obs]


Output()

KeyError: 'y_obs'

In [12]:
# Display available keys in posterior predictive samples
print("Available keys in posterior predictive sample:", post_pred.keys())


Available keys in posterior predictive sample: KeysView(Inference data with groups:
	> posterior_predictive
	> observed_data)


In [17]:
# Ensure posterior predictive samples and retrieve predictions
with bayesian_model:
    post_pred = pm.sample_posterior_predictive(trace)

# Retrieve y_pred_samples as a flattened array if necessary
y_pred_samples = post_pred['posterior_predictive']['y_obs']

# Trim or expand y_test to match y_pred_samples shape
n_samples = y_pred_samples.shape[0]
y_test_adjusted = y_test[:n_samples]

# Calculate mean prediction across samples
y_pred = y_pred_samples.mean(axis=0)

# Check for length consistency before calculating metrics
if len(y_test_adjusted) == len(y_pred):
    # Performance metrics using the adjusted y_test
    mse = mean_squared_error(y_test_adjusted, y_pred)
    mae = mean_absolute_error(y_test_adjusted, y_pred)
    r2 = r2_score(y_test_adjusted, y_pred)

    print(f"Bayesian Model - Mean Squared Error (MSE): {mse}")
    print(f"Bayesian Model - Mean Absolute Error (MAE): {mae}")
    print(f"Bayesian Model - R-squared (R²): {r2}")
else:
    print("Error: y_test and y_pred lengths are still inconsistent after adjustment.")


Sampling: [y_obs]


Output()

Error: y_test and y_pred lengths are still inconsistent after adjustment.


In [18]:
# Ensure posterior predictive samples
with bayesian_model:
    post_pred = pm.sample_posterior_predictive(trace)

# Retrieve y_pred_samples and y_test to check their shapes and lengths
y_pred_samples = post_pred['posterior_predictive']['y_obs']  # Adjust the key name if necessary

# Print shapes and lengths for debugging
print("Shape of y_pred_samples:", y_pred_samples.shape)
print("Length of y_pred_samples (flattened):", y_pred_samples.size)
print("Shape of y_test:", y_test.shape)
print("Length of y_test:", len(y_test))

# Display first few values to further inspect if needed
print("\nFirst few values in y_pred_samples:", y_pred_samples[:5])
print("First few values in y_test:", y_test[:5])


Sampling: [y_obs]


Output()

Shape of y_pred_samples: (2, 500, 2912)
Length of y_pred_samples (flattened): 2912000
Shape of y_test: (728,)
Length of y_test: 728

First few values in y_pred_samples: <xarray.DataArray 'y_obs' (chain: 2, draw: 500, y_obs_dim_2: 2912)> Size: 23MB
array([[[3.30690686, 1.95536389, 3.19558551, ..., 2.17847187,
         3.03065838, 2.07913835],
        [2.67406683, 3.08800952, 1.35047075, ..., 1.51752574,
         2.22007913, 1.38032075],
        [3.32318334, 3.47987353, 2.02704247, ..., 2.81532151,
         2.05349423, 1.21523731],
        ...,
        [2.96711413, 2.4549714 , 1.23077835, ..., 2.45742247,
         2.42744784, 2.2728746 ],
        [1.94756812, 2.71131039, 2.16711761, ..., 2.52207523,
         2.05910402, 2.12224614],
        [2.7489549 , 2.44574212, 1.97507451, ..., 2.19150109,
         2.41176208, 1.78427977]],

       [[2.82392109, 1.99720588, 1.85623253, ..., 2.0576151 ,
         1.99541197, 2.30666961],
        [2.76388329, 2.54667583, 1.51051441, ..., 2.17321307,
   

In [20]:
# Redefine posterior predictive sampling, focused on X_test
with bayesian_model:
    # Pass in `X_test` for posterior predictions only for the test set
    post_pred_test = pm.sample_posterior_predictive(trace, var_names=['y_obs'], random_seed=42)

# Select the dimension for y_test in predictions and calculate the mean across chains and draws
y_pred_samples = post_pred_test['posterior_predictive']['y_obs']
y_pred = y_pred_samples.mean(axis=(0, 1))[:len(y_test)]  # Mean over chains and draws

# Performance metrics - matching length to y_test
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Bayesian Model - Mean Squared Error (MSE): {mse}")
print(f"Bayesian Model - Mean Absolute Error (MAE): {mae}")
print(f"Bayesian Model - R-squared (R²): {r2}")


Sampling: [y_obs]


Output()

Bayesian Model - Mean Squared Error (MSE): 0.16286788909102032
Bayesian Model - Mean Absolute Error (MAE): 0.31773397915293367
Bayesian Model - R-squared (R²): -0.2749402148627538
